In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-prime-movies-and-tv-shows/amazon_prime_titles.csv
/kaggle/input/netflix-shows/netflix_titles.csv
/kaggle/input/disney-movies-and-tv-shows/disney_plus_titles.csv


Here we will be building content based recommendation system for movies and web shows on 3 OTT platforms viz: Netflix,
Amazon Prime and Disney+ Hotstar.
As these are most popular OTT platforms worldwide, one might stuck on what to watch on them.

# Loading Datasets

In [2]:
# Load netflix data
netflix = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
netflix.shape

(8807, 12)

In [4]:
# Load Amazon Prime Data
amznprime = pd.read_csv("/kaggle/input/amazon-prime-movies-and-tv-shows/amazon_prime_titles.csv")
amznprime.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [5]:
amznprime.shape

(9668, 12)

In [6]:
hotstar = pd.read_csv("/kaggle/input/disney-movies-and-tv-shows/disney_plus_titles.csv")
hotstar.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [7]:
hotstar.shape

(1450, 12)

We now concatinate all three dataframes as we are building combined recommendation system.
* column "show id" is common in all dataframes, we will drop it 
* In final dataframe we want information about on which platform recommended shows are available
* We will add one column naming "platform" in all dataframes,for example :- **netflix['platform']** containing all values as "Netflix" 

In [8]:
# Dropping "show_id" column
netflix.drop("show_id", axis=1, inplace=True)
amznprime.drop("show_id",axis=1, inplace=True)
hotstar.drop("show_id", axis=1, inplace=True)

In [9]:
# adding "platform" column to respective dataframe

netflix['platform'] = "Netflix"
amznprime['platform'] = "Amazon Prime"
hotstar['platform'] = "Disney+ Hotstar"

In [10]:
## creating final Dataframe
df = pd.concat([netflix, amznprime, hotstar], ignore_index=True)
df.shape

(19925, 12)

In [11]:
df.tail()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform
19920,Movie,X-Men Origins: Wolverine,Gavin Hood,"Hugh Jackman, Liev Schreiber, Danny Huston, wi...","United States, United Kingdom","June 4, 2021",2009,PG-13,108 min,"Action-Adventure, Family, Science Fiction",Wolverine unites with legendary X-Men to fight...,Disney+ Hotstar
19921,Movie,Night at the Museum: Battle of the Smithsonian,Shawn Levy,"Ben Stiller, Amy Adams, Owen Wilson, Hank Azar...","United States, Canada","April 2, 2021",2009,PG,106 min,"Action-Adventure, Comedy, Family",Larry Daley returns to rescue some old friends...,Disney+ Hotstar
19922,Movie,Eddie the Eagle,Dexter Fletcher,"Tom Costello, Jo Hartley, Keith Allen, Dickon ...","United Kingdom, Germany, United States","December 18, 2020",2016,PG-13,107 min,"Biographical, Comedy, Drama","True story of Eddie Edwards, a British ski-jum...",Disney+ Hotstar
19923,Movie,Bend It Like Beckham,Gurinder Chadha,"Parminder Nagra, Keira Knightley, Jonathan Rhy...","United Kingdom, Germany, United States","September 18, 2020",2003,PG-13,112 min,"Buddy, Comedy, Coming of Age",Despite the wishes of their traditional famili...,Disney+ Hotstar
19924,Movie,Captain Sparky vs. The Flying Saucers,Mark Waring,Charlie Tahan,United States,"April 1, 2020",2012,TV-G,2 min,"Action-Adventure, Animals & Nature, Animation",View one of Sparky's favorite home movies.,Disney+ Hotstar


# Analysis of final Dataframe

In [12]:
# Type of content available
df.type.unique()

array(['Movie', 'TV Show'], dtype=object)

In [13]:
# misssing values
df.isnull().sum()

type                0
title               0
director         5189
cast             2248
country         10046
date_added       9526
release_year        0
rating            344
duration            3
listed_in           0
description         0
platform            0
dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19925 entries, 0 to 19924
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          19925 non-null  object
 1   title         19925 non-null  object
 2   director      14736 non-null  object
 3   cast          17677 non-null  object
 4   country       9879 non-null   object
 5   date_added    10399 non-null  object
 6   release_year  19925 non-null  int64 
 7   rating        19581 non-null  object
 8   duration      19922 non-null  object
 9   listed_in     19925 non-null  object
 10  description   19925 non-null  object
 11  platform      19925 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.8+ MB


In [15]:
# we need to create new Data frame according to features required for recommendation
newdf = df.drop('rating', axis=1)
newdf.head()

,type,title,director,cast,country,date_added,release_year,duration,listed_in,description,platform
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,90 min,Documentaries,"As her father nears the end of his life, filmm...",Netflix
1,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Netflix
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Netflix
3,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Netflix
4,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Netflix


In [16]:
dropped_nan = df.dropna()
dropped_nan.shape

(6151, 12)

We have large number of entries with missing values
dropping them will result in loss of information

In [17]:
# removing spaces between individual terms in entries
# for ex ""
newdf['director']=newdf['director'].apply(lambda x: str(x).replace(" ",""))

In [18]:
newdf['cast']=newdf['cast'].apply(lambda x: str(x).replace(" ",""))
newdf['country']=newdf['country'].apply(lambda x: str(x).replace(" ",""))
newdf['listed_in']=newdf['listed_in'].apply(lambda x: str(x).replace(" ",""))

In [19]:
newdf['type']=newdf['type'].apply(lambda x: str(x).replace(" ",""))

In [20]:
# creating list of words in each string available in 'description' column

newdf['description']=newdf['description'].apply(lambda x:x.split())

In [21]:
# converting all rows to list
newdf['type']=newdf['type'].apply(lambda x:list(x.split(" ")))
newdf['listed_in']=newdf['listed_in'].apply(lambda x:list(x.split(" ")))
newdf['director'] = newdf['director'].apply(lambda x:list(x.split(" ")))
newdf['cast'] = newdf['cast'].apply(lambda x:list(x.split(" ")))
newdf['country'] = newdf['country'].apply(lambda x:list(x.split(" ")))
newdf['release_year'] =newdf['release_year'].apply(lambda x:list(str(x).split(" ")))

In [22]:
# creating new column for tags
newdf['tags'] = newdf['type']+newdf['director']+newdf['cast']+newdf['country']+newdf['release_year']+newdf['listed_in']+newdf['description']

In [23]:
# we have lot of entries with "nan" values, remove those values from tag list
for row in newdf['tags']:
    if "nan" in row:
        row.remove("nan")

In [24]:
# to form vectors we need strings, so convert list of tags to string
newdf['tags']=newdf['tags'].apply(lambda x:" ".join(x))

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
# create instance of TFIDF vectorizer with stopwords in "english" language 
#so that vectors will be formed without frequent stopwords
tv = TfidfVectorizer(max_features=30000,stop_words="english")

In [27]:
# create vectors and storing them into array
vectors = tv.fit_transform(newdf['tags']).toarray()

In [28]:
vectors.shape

(19925, 30000)

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
similarity

array([[1.        , 0.        , 0.        , ..., 0.0086881 , 0.00875237,
        0.01567835],
       [0.        , 1.        , 0.03418614, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03418614, 1.        , ..., 0.        , 0.        ,
        0.0183939 ],
       ...,
       [0.0086881 , 0.        , 0.        , ..., 1.        , 0.05630591,
        0.01206159],
       [0.00875237, 0.        , 0.        , ..., 0.05630591, 1.        ,
        0.01215082],
       [0.01567835, 0.        , 0.0183939 , ..., 0.01206159, 0.01215082,
        1.        ]])

In [30]:
dis=sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])
dis[1:6]

[(4877, 0.18518186605972042),
 (13941, 0.15782719075269164),
 (19821, 0.13629445714417657),
 (16175, 0.13616883177317995),
 (6038, 0.1355774306523915)]

In [31]:
#newdf[newdf['title'] == "Ganglands"].index 
# above code gives tuple => Int64Index([2], dtype='int64')
# we jusst want index [2] to be fetched, in 0th position in tuple

newdf[newdf['title'] == "Ganglands"].index[0]

2

In [32]:
# define recommend function

def recommend(movie):
    index = newdf[newdf['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True,key=lambda x:x[1])
    #for i in distances[1:6]:
        #print(newdf.iloc[i[0]].title)
        #print("Watch on:- ",newdf.iloc[i[0]].platform)
    dic = {0: df.iloc[distances[1][0]],
           1: df.iloc[distances[2][0]],
           2: df.iloc[distances[3][0]],
           3: df.iloc[distances[4][0]],
           4: df.iloc[distances[5][0]]}
    df1 = pd.DataFrame(dic).T
    return df1

In [33]:
recommend("Kota Factory")

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform
0,TV Show,Yeh Meri Family,NaN,"Vishesh Bansal, Mona Singh, Akarsh Khurana, Ah...",India,"August 31, 2018",2018,TV-PG,1 Season,"International TV Shows, TV Comedies","In the summer of 1998, middle child Harshu bal...",Netflix
1,TV Show,Girls Hostel,NaN,"Srishti Shrivastava, Parul Gulati, Simran Nate...",India,"October 1, 2019",2018,TV-MA,1 Season,"International TV Shows, TV Comedies, TV Dramas",Four girls from different backgrounds become u...,Netflix
2,Movie,Luckee,Sanjay Jadhav,"Abhay Mahajan, Dipti Sati, Mayur More, Shubhan...",India,"June 15, 2019",2019,TV-14,113 min,"Comedies, International Movies, Romantic Movies",When his friends make a bet that he can woo hi...,Netflix
3,TV Show,Little Things,NaN,"Dhruv Sehgal, Mithila Palkar",India,"November 9, 2019",2019,TV-MA,3 Seasons,"International TV Shows, Romantic TV Shows, TV ...",A cohabiting couple in their 20s navigate the ...,Netflix
4,TV Show,Titipo Titipo,NaN,NaN,NaN,NaN,2021,ALL,1 Season,"Animation, Kids",Titipo is the new train who just became offici...,Amazon Prime
